In [1]:
# Import required packages
import os
import sys
import getpass
import asf_search as asf
import logging
import isce
root_logger = logging.getLogger()
root_logger.setLevel('WARNING')
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from datetime import datetime, timedelta
import xarray as xr
import rasterio as rio
import rioxarray
import geopandas as gpd
import time
from glob import glob
import scipy.signal

In [ ]:
# # install autoRIFT in notebook environment
# !mamba install --yes --prefix {sys.prefix} autorift
# !{sys.executable} -m pip install opencv-python-headless

In [2]:
from autoRIFT import autoRIFT

In [3]:
# Set environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)

## Define utilities

In [4]:
# select pairs from pair list
def select_pairs(scene_list, max_temp_bline):
    scene_dates = {}
    for scene in scene_list:
        date = scene[17:25]
        scene_dates[date] = scene
        
    pair_dict = {}
    pair_scenes = []
    for date1 in scene_dates:
        for date2 in scene_dates:
            if datetime.strptime(date2, '%Y%m%d')-datetime.strptime(date1, '%Y%m%d') < timedelta(days=max_temp_bline) and not date1 >= date2 :
                pair_dict[f'{date1}-{date2}'] = [scene_dates[date1], scene_dates[date2]]
                pair_scenes.append(scene_dates[date1])
                pair_scenes.append(scene_dates[date2])
    pair_scenes = [*set(pair_scenes)]
    
    print(f'number of pairs: {len(pair_dict)}')
    
    return pair_dict, pair_scenes

In [5]:
def generate_configs(pair_dict,
                     proc_path,
                     range_looks,
                     azimuth_looks,
                     aoi):
    
    for pair in pair_dict:
        os.chdir(f'{proc_path}/{pair}')

        reference = pair_dict[pair][0]
        secondary = pair_dict[pair][1]

        cmd_topsApp_config =f'''<?xml version="1.0" encoding="UTF-8"?>
        <topsApp>
          <component name="topsinsar">
            <property name="Sensor name">SENTINEL1</property>
            <component name="reference">
                <catalog>reference.xml</catalog>
            </component>
            <component name="secondary">
                <catalog>secondary.xml</catalog>
            </component>
            <property name="range looks">{range_looks}</property>
            <property name="azimuth looks">{azimuth_looks}</property>
            <property name="region of interest">{aoi}</property>
            <property name="do unwrap">True</property>
            <property name="unwrapper name">snaphu_mcf</property>
            <property name="do denseoffsets">True</property>
            <property name="geocode list">['merged/phsig.cor', 'merged/filt_topophase.unw', 'merged/los.rdr', 'merged/topophase.flat', 'merged/filt_topophase.flat','merged/topophase.cor','merged/filt_topophase.unw.conncomp']</property>
          </component>
        </topsApp>'''
        print("writing topsApp.xml")
        with open("topsApp.xml", "w") as fid:
            fid.write(cmd_topsApp_config)

        cmd_reference_config = f'''<component name="reference">
            <property name="orbit directory">{proc_path}/orbits</property>
            <property name="output directory">reference</property>
            <property name="safe">['{proc_path}/slc/{reference}.zip']</property>
        </component>'''
        print("writing reference.xml")
        with open("reference.xml", "w") as fid:
            fid.write(cmd_reference_config)

            cmd_secondary_config = f'''<component name="secondary">
            <property name="orbit directory">{proc_path}/orbits</property>
            <property name="output directory">secondary</property>
            <property name="safe">['{proc_path}/slc/{secondary}.zip']</property>
        </component>'''
        print("writing secondary.xml")
        with open("secondary.xml", "w") as fid:
            fid.write(cmd_secondary_config)

In [6]:
def isce_to_ds(path):
    
    unw_phase_path = glob(f'{path}/merged/filt_topophase.unw.geo.vrt')[0]
    dem_path = glob(f'{path}/merged/dem.crop.vrt')[0]
    w_phase_path = glob(f'{path}/merged/filt_topophase.flat.geo.vrt')[0]
    corr_path = glob(f'{path}/merged/topophase.cor.geo.vrt')[0]
    conncomp_path = glob(f'{path}/merged/filt_topophase.unw.conncomp.geo.vrt')[0]
    
    ds = rioxarray.open_rasterio(unw_phase_path).isel(band=1).to_dataset(name='unw_phase')
    dem_da = rioxarray.open_rasterio(dem_path)
    w_phase_da = rioxarray.open_rasterio(w_phase_path)
    corr_da = rioxarray.open_rasterio(corr_path).isel(band=1)
    conncomp_da = rioxarray.open_rasterio(conncomp_path)
    
    ds['elevation'] = (('y', 'x'), dem_da.squeeze().values)
    ds['w_phase'] = (('y', 'x'), np.angle(w_phase_da.squeeze()))
    ds['coherence'] = (('y', 'x'), corr_da.squeeze().values)
    ds['conncomp'] = (('y', 'x'), conncomp_da.squeeze().values)
    
    ds = ds.rio.reproject(ds.rio.estimate_utm_crs())
    
    return ds

In [7]:
# open coregistered SLCs
def isce_to_np(path):
    unw_phase_path = glob(f'{path}/merged/reference.slc.full.vrt')[0]
    dem_path = glob(f'{path}/merged/secondary.slc.full.vrt')[0]
    
    ref_np = np.abs(rioxarray.open_rasterio(unw_phase_path).squeeze().values)
    sec_np = np.abs(rioxarray.open_rasterio(dem_path).squeeze().values)
    
    return ref_np, sec_np

In [8]:
def prep_autorift(ref, sec):
    # setup autoRIFT parameters
    obj = autoRIFT()
    obj.I1 = ref
    obj.I2 = sec

    # define chip size ranges
    ## Kernel sizes to use for correlation
    obj.ChipSizeMinX = 16
    obj.ChipSizeMaxX = 128

    # check and refine this
    obj.ChipSize0X = 16

    # skip rate setup
    # Produce output at a grid x time the input resolution 
    obj.SkipSampleX = 4
    obj.SkipSampleY = 4

    # preprocess to bring out edges in the image
    prefilter_choice = 'lap'
    obj.WallisFilterWidth = 3
    obj.preprocess_filt_wal()
    
    # downcast to 8 bit
    obj.uniform_data_type()

    # create the grid if it does not exist (from test_autoRIFT.py)
    m,n = obj.I1.shape
    xGrid = np.arange(obj.SkipSampleX+10,n-obj.SkipSampleX,obj.SkipSampleX)
    yGrid = np.arange(obj.SkipSampleY+10,m-obj.SkipSampleY,obj.SkipSampleY) 
    nd = xGrid.__len__()
    md = yGrid.__len__()
    obj.xGrid = np.int32(np.dot(np.ones((md,1)),np.reshape(xGrid,(1,xGrid.__len__()))))
    obj.yGrid = np.int32(np.dot(np.reshape(yGrid,(yGrid.__len__(),1)),np.ones((1,nd))))
    noDataMask = np.logical_not(obj.xGrid)
    
    return obj

In [9]:
def offsets_to_isce(obj, path):
    
    # save to isce rasters
    azimuth_offsets = xr.DataArray(obj.Dy,
                                   coords={'y':np.arange(obj.Dy.shape[0], 0, -1), 'x':np.arange(0, obj.Dy.shape[1])},
                                   dims=['y', 'x'])

    range_offsets = xr.DataArray(obj.Dx,
                                 coords={'y':np.arange(obj.Dx.shape[0], 0, -1), 'x':np.arange(0, obj.Dx.shape[1])},
                                 dims=['y', 'x'])

    azimuth_offsets.rio.to_raster('./merged/azimuth_offsets_raw', driver='ISCE')
    range_offsets.rio.to_raster('./merged/range_offsets_raw', driver='ISCE')
    
    # build vrts and multilook offsets
    !gdalbuildvrt $f'{path}/merged/azimuth_offsets_raw.vrt' $f'{path}/merged/azimuth_offsets_raw'
    !looks.py -i $f'{path}/merged/azimuth_offsets_raw' -r 7 -a 3 -o $f'{path}/merged/azimuth_offsets_raw.lks'
    !gdalbuildvrt $f'{path}/merged/range_offsets_raw.vrt' $f'{path}/merged/range_offsets_raw'
    !looks.py -i $f'{path}/merged/range_offsets_raw' -r 7 -a 3 -o $f'{path}/merged/range_offsets_raw.lks'

In [10]:
def offsets_to_phase(path):
    w_phase_path = glob(f'{path}/merged/filt_topophase.flat.vrt')[0]
    range_offsets_path = glob(f'{path}/merged/range_offsets_raw.lks.vrt')[0]
    
    ds = rioxarray.open_rasterio(w_phase_path).to_dataset(name='w_phase').squeeze()
    ds['w_phase'] = (('y', 'x'), np.angle(ds.w_phase.squeeze()))
    
    ro_da = rioxarray.open_rasterio(range_offsets_path).squeeze()
    
    # grab coord ranges from wrapped phase
    ro_da = ro_da.assign_coords(x=np.linspace(ds.w_phase.x.min(), ds.w_phase.x.max(), num= len(ro_da.x)),
                                y=np.linspace(ds.w_phase.y.min(), ds.w_phase.y.max(), num= len(ro_da.y)))
    
    ds['range_offsets'] = (('y', 'x'), ro_da.interp_like(ds.w_phase).values)
    
    # convert range offset to phase. pixel spacing=2.3
    ds['range_offsets_phase'] = (ds.range_offsets*2.3*4*np.pi)/0.05546576
    
    
    ds.assign_coords(y=np.flip(ds.y.values)).range_offsets_phase.rio.to_raster('./merged/range_offsets_phase', driver='ISCE')
    !gdalbuildvrt $f'{path}/merged/range_offsets_phase.vrt' $f'{path}/merged/range_offsets_phase'

## Data download

In [ ]:
# all scenes from 2021
# scene_list=['S1A_IW_SLC__1SDV_20200104T001127_20200104T001154_030643_0382F8_6A1D',
#             'S1A_IW_SLC__1SDV_20200116T001126_20200116T001153_030818_038917_3062',
#             'S1A_IW_SLC__1SDV_20200128T001126_20200128T001153_030993_038F41_DBA0',
#             'S1A_IW_SLC__1SDV_20200209T001126_20200209T001153_031168_03955D_675E',
#             'S1A_IW_SLC__1SDV_20200221T001125_20200221T001152_031343_039B60_5143',
#             'S1A_IW_SLC__1SDV_20200304T001125_20200304T001152_031518_03A16E_C0D7',
#             'S1A_IW_SLC__1SDV_20200316T001126_20200316T001153_031693_03A77D_F557',
#             'S1A_IW_SLC__1SDV_20200328T001126_20200328T001153_031868_03ADA6_67A1',
#             'S1A_IW_SLC__1SDV_20200409T001126_20200409T001153_032043_03B3D4_B4A4',
#             'S1A_IW_SLC__1SDV_20200421T001126_20200421T001153_032218_03B9F9_6754',
#             'S1A_IW_SLC__1SDV_20200503T001127_20200503T001154_032393_03C021_B585',
#             'S1A_IW_SLC__1SDV_20200515T001128_20200515T001155_032568_03C5AC_BEB7',
#             'S1A_IW_SLC__1SDV_20200527T001128_20200527T001155_032743_03CAFC_0FFF',
#             'S1A_IW_SLC__1SDV_20200608T001129_20200608T001156_032918_03D024_6055',
#             'S1A_IW_SLC__1SDV_20200620T001130_20200620T001157_033093_03D56E_D0A5',
#             'S1A_IW_SLC__1SDV_20a200702T001130_20200702T001157_033268_03DABD_8873',
#             'S1A_IW_SLC__1SDV_20200714T001131_20200714T001158_033443_03E012_0803',
#             'S1A_IW_SLC__1SDV_20200726T001132_20200726T001159_033618_03E572_D723',
#             'S1A_IW_SLC__1SDV_20200807T001133_20200807T001200_033793_03EAE9_EE4D',
#             'S1A_IW_SLC__1SDV_20200819T001133_20200819T001200_033968_03F110_A161',
#             'S1A_IW_SLC__1SDV_20200831T001134_20200831T001201_034143_03F73A_BA33',
#             'S1A_IW_SLC__1SDV_20200912T001135_20200912T001201_034318_03FD5A_32AE',
#             'S1A_IW_SLC__1SDV_20200924T001135_20200924T001202_034493_04038B_D1A6',
#             'S1A_IW_SLC__1SDV_20201006T001135_20201006T001202_034668_0409B2_0D91',
#             'S1A_IW_SLC__1SDV_20201018T001133_20201018T001159_034843_040FD6_7A19',
#             'S1A_IW_SLC__1SDV_20201030T001133_20201030T001159_035018_0415C8_9680',
#             'S1A_IW_SLC__1SDV_20201111T001132_20201111T001159_035193_041BE5_06B2',
#             'S1A_IW_SLC__1SDV_20201123T001132_20201123T001159_035368_0421EE_C267',
#             'S1A_IW_SLC__1SDV_20201205T001132_20201205T001158_035543_0427F0_9243',
#             'S1A_IW_SLC__1SDV_20201217T001131_20201217T001158_035718_042E06_027E',
#             'S1A_IW_SLC__1SDV_20201229T001131_20201229T001157_035893_04341E_E3B4'
#            ]

In [11]:
scene_list=['S1A_IW_SLC__1SDV_20201006T001135_20201006T001202_034668_0409B2_0D91',
            'S1A_IW_SLC__1SDV_20201018T001133_20201018T001159_034843_040FD6_7A19',
            'S1A_IW_SLC__1SDV_20201030T001133_20201030T001159_035018_0415C8_9680',
            'S1A_IW_SLC__1SDV_20201111T001132_20201111T001159_035193_041BE5_06B2',
            'S1A_IW_SLC__1SDV_20201123T001132_20201123T001159_035368_0421EE_C267',
            'S1A_IW_SLC__1SDV_20201205T001132_20201205T001158_035543_0427F0_9243',
            'S1A_IW_SLC__1SDV_20201217T001131_20201217T001158_035718_042E06_027E',
            'S1A_IW_SLC__1SDV_20201229T001131_20201229T001157_035893_04341E_E3B4'
           ]

In [12]:
pair_dict, pair_scenes = select_pairs(scene_list, 80)

number of pairs: 27


In [13]:
proc_path = '/home/jovyan/ffits/nbs/imja/proc_DT121'

for pair in pair_dict:
    pair_path = f'{proc_path}/{pair}'
    if not os.path.exists(pair_path):
        os.makedirs(pair_path)

In [ ]:
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

In [ ]:
# download slcs
os.chdir(f'{proc_path}/slc')

results = asf.granule_search(pair_scenes)
session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
results.download(path=f'{proc_path}/slc', processes=2, session=session)

In [ ]:
# download orbits
os.chdir(f'{proc_path}/orbits')

In [ ]:
%%bash
wget -nc https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
chmod +x fetchOrbit.py

In [ ]:
# grab orbital files with fetchOrbit.py
for scene in pair_scenes:
    os.system(f'./fetchOrbit.py -i {scene}')

In [ ]:
generate_configs(pair_dict=pair_dict,
                proc_path=proc_path,
                range_looks=7,
                azimuth_looks=3,
                aoi=[27.80, 28.13, 86.28, 87.32])

## run isce and autorift

In [14]:
def proc_pairs(pair_dict, proc_path):
    for i, pair in enumerate(pair_dict):
        print(f'###################### working on {pair}, {i+1}/{len(pair_dict)} ######################')
        os.chdir(f'{proc_path}/{pair}')
        if not os.path.exists(f'{proc_path}/{pair}/merged/filt_topophase.unw.geo'):
            print('###################### running isce ######################')
            !topsApp.py --start=preprocess --end=geocode
        if not os.path.exists(f'{proc_path}/{pair}/merged/range_offsets_phase.vrt'):
            ref, sec = isce_to_np(f'{proc_path}/{pair}')
            obj = prep_autorift(ref, sec)
            print('###################### running autorift ######################')
            obj.runAutorift()
            offsets_to_isce(obj, f'{proc_path}/{pair}')
            offsets_to_phase(f'{proc_path}/{pair}')

In [15]:
proc_pairs(pair_dict, proc_path)

###################### working on 20201006-20201018, 1/27 ######################
###################### working on 20201006-20201030, 2/27 ######################
###################### working on 20201006-20201111, 3/27 ######################
###################### working on 20201006-20201123, 4/27 ######################
###################### working on 20201006-20201205, 5/27 ######################
###################### working on 20201006-20201217, 6/27 ######################
###################### working on 20201018-20201030, 7/27 ######################
###################### working on 20201018-20201111, 8/27 ######################
###################### working on 20201018-20201123, 9/27 ######################
###################### working on 20201018-20201205, 10/27 ######################
###################### working on 20201018-20201217, 11/27 ######################
###################### working on 20201018-20201229, 12/27 ######################
###################### wo